In [10]:
import numpy as np
import rasterio
import geopandas as gpd
from rasterio.mask import mask
import os
import pandas as pd

In [15]:
def layer_alterator(
    raster_folder,
    vector_path,
    operation_rule,
    output_folder=None,
    value_range=(0, 1)
):
    """
    Modifies raster pixels using vector-defined zones and attribute-based operations.

    Parameters:
    - raster_folder (str): Folder containing rasters (supports subfolders).
    - vector_path (str): Path to vector mask (.geojson or .gpkg).
    - operation_rule (str): Either 'increase_pct' or 'reduction'.
    - output_folder (str): Where to save the results.
    - value_range (tuple): Min/max clip range for pixel values.
    """

    mask_gdf = gpd.read_file(vector_path)

    # Collect all .tif files recursively
    raster_files = []
    for root, dirs, files in os.walk(raster_folder):
        for file in files:
            if file.lower().endswith((".tif", ".tiff")):
                raster_files.append(os.path.join(root, file))

    for raster_path in raster_files:
        raster_name = os.path.splitext(os.path.basename(raster_path))[0]
        attribute_field = raster_name.upper()  # Attribute names are like F_AC, BH, etc.

        if attribute_field not in mask_gdf.columns:
            print(f"[SKIP] Attribute '{attribute_field}' not found in vector data.")
            continue

        with rasterio.open(raster_path) as src:
            raster_data = src.read(1)
            transform = src.transform
            crs = src.crs
            dtype = src.dtypes[0]

            for _, row in mask_gdf.iterrows():
                if pd.isna(row.get(attribute_field)):
                    continue

                try:
                    value = float(row[attribute_field])
                except (ValueError, TypeError):
                    print(f"[SKIP] Invalid value for '{attribute_field}': {row[attribute_field]}")
                    continue

                geometry = [row["geometry"]]

                # Mask the raster using the current geometry
                masked_data, masked_transform = mask(src, geometry, crop=False)
                masked_data = masked_data[0]

                polygon_mask, _ = mask(src, geometry, crop=False)
                polygon_mask = polygon_mask[0]

                # Apply the operation
                if operation_rule == "increase_pct":
                    modified = masked_data + (masked_data * (value / 100.0))
                elif operation_rule == "reduction":
                    modified = masked_data - value
                else:
                    print(f"[ERROR] Unknown operation: {operation_rule}")
                    continue

                modified = np.clip(modified, *value_range)
                raster_data[polygon_mask > 0] = modified[polygon_mask > 0]

        # Save the modified raster
        if output_folder:
            os.makedirs(output_folder, exist_ok=True)
            output_path = os.path.join(output_folder, os.path.basename(raster_path))
        else:
            output_path = f"modified_{os.path.basename(raster_path)}"

        with rasterio.open(
            output_path, "w", driver="GTiff",
            height=raster_data.shape[0], width=raster_data.shape[1],
            count=1, dtype=dtype, crs=crs, transform=transform
        ) as dst:
            dst.write(raster_data, 1)

        print(f"[DONE] Modified {os.path.basename(raster_path)} → {output_path}")

In [16]:
layer_alterator(
    raster_folder="./test_data/test_data_final/lc_fractions",
    vector_path="./test_data/test_data_final/sample_mask.geojson",  # now accepts GeoJSON or GPKG
    output_folder="./output/modified_raster/new_data",
    operation_rule="increase_pct",
    value_range=(0, 1)
)

[DONE] Modified F_W.tif → ./output/modified_raster/new_data/F_W.tif
[DONE] Modified F_S.tif → ./output/modified_raster/new_data/F_S.tif
[DONE] Modified F_G.tif → ./output/modified_raster/new_data/F_G.tif
[DONE] Modified F_BS.tif → ./output/modified_raster/new_data/F_BS.tif
[DONE] Modified F_TV.tif → ./output/modified_raster/new_data/F_TV.tif
[DONE] Modified F_AC.tif → ./output/modified_raster/new_data/F_AC.tif
[DONE] Modified F_M.tif → ./output/modified_raster/new_data/F_M.tif
